# Regional Tasmania forced by GLORYS and ERA5 reanalysis datasets

**Note: FRE-NC tools are required to be set up, as outlined in the `regional-mom6` package [documentation](https://regional-mom6.readthedocs.io/en/latest/).**

For this example we need a copy of the [GEBCO bathymetry](https://www.gebco.net/data_and_products/gridded_bathymetry_data/), access to the [GLORYs ocean reanalysis data](https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/description), and [ERA5 surface forcing](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5). 

This example reads in the entire global extent of ERA5 and GEBCO; we don't need to worry about cutting it down to size. 

## What does this notebook do?
This notebook is designed to set you up with a working MOM6 regional configuration. First, try and get it running with our default Tasmania case, then you can clone the notebook and modify for your region of interest. 

Input Type | Source | Subsets required
---|---|---
Surface | [ERA5 surface forcing](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) | Data from 2003; whole globe or subset around our domain
Ocean | [GLORYs reanalysis product](https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/description) | Boundary segments & initial condition; see section 2 for details.   
Bathymetry | [GEBCO](https://www.gebco.net/data_and_products/gridded_bathymetry_data/) | whole globe or subset around domain

In [3]:
import os
import regional_mom6 as rmom6
from pathlib import Path
from dask.distributed import Client
client = Client() # Start a dask cluster

## Step 1: Choose our domain, define workspace paths

You can use the <a href="https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/download" >graphical user interface (GUI) by Copernicus</a> to find the latitude-longitude extent of the domain of your interest.

In [4]:
expt_name = "tasmania-example-reanalysis"

latitude_extent = [-48, -38.95]
longitude_extent = [143, 150]

date_range = ["2003-01-01 00:00:00", "2003-01-05 00:00:00"]

## Place where all your input files go 
input_dir = Path(f"mom6_input_directories/{expt_name}/")

## Directory where you'll run the experiment from
run_dir = Path(f"mom6_run_directories/{expt_name}/")

## Directory where fre tools are stored 
toolpath_dir = Path("PATH_TO_FRE_TOOLS") ## Compiled tools needed for construction of mask tables

## Path to where your raw ocean forcing files are stored
glorys_path = Path("PATH_TO_GLORYS_DATA" )

for i in [run_dir, glorys_path, input_dir]:
    if not os.path.exists(str(i)):
        os.makedirs(str(i))

## Step 2: Prepare ocean forcing data

We need to cut out our ocean forcing. The package expects an initial condition and one time-dependent segment per non-land boundary. Naming convention is "east_unprocessed" for segments and "ic_unprocessed" for the initial condition.

Data can be downloaded directly from the [Copernicus Marine data store](https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/download) via their GUI (once logged in).

1. Initial condition: Using the GUI, select an area matching your `longitude_extent` and `latitude_extent` that corresponds to the first day in your date range. Download the initial condition and save it with filename `ic_unprocessed.nc` inside the `glorys_path` directory.
2. Boundary forcing: Using the GUI, select the Eastern boundary of your domain (if you have one that contains ocean). Allow for a buffer of ~0.5 degrees in all directions, and download for the prescribed `date_range`. Download and name `east_unprocessed.nc`.
3. Repeat step 2 for the rest sections of the domain.

## Step 3: Make experiment object
The `regional_mom6.experiment` contains the regional domain basics, as well as it generates the horizontal and vertical grids, `hgrid` and `vgrid` respectively, and sets up the directory structures. 




In [ ]:
expt = rmom6.experiment(
    longitude_extent = longitude_extent,
    latitude_extent = latitude_extent,
    date_range = date_range,
    resolution = 0.05,
    number_vertical_layers = 75,
    layer_thickness_ratio = 10,
    depth = 4500,
    mom_run_dir = run_dir,
    mom_input_dir = input_dir,
    toolpath_dir = toolpath_dir
)

We can now access the horizontal and vertical grid of the regional configuration via `expt.hgrid` and `expt.vgrid` respectively.

By plotting `vgrid` with `marker = '.'` option allows us to inspect the vertical spacing. Alternatively,

```python
np.diff(expt.vgrid.zl).plot(marker = '.')
```
shows you the vertical spacing profile.

### Modular workflow!

After constructing our `experiment` object, if for some reason we are not happy with the grids, then we can simply modify them and then save them back into the `experiment` object. However, in doing so, we also need to save them to disk again. For example:

```python
new_hgrid = xr.open_dataset(inputdir / "hgrid.nc")
```
Modify `new_hgrid`, ensuring that _all metadata_ is retained to keep MOM6 happy. Then, save any changes

```python
expt.hgrid = new_hgrid

expt.hgrid.to_netcdf(inputdir / "hgrid.nc")
```

## Step 4: Set up bathymetry

Similarly to ocean forcing, we point the experiment's `bathymetry` method at the location of the file of choice and also pass a dictionary mapping variable names. We don't don't need to preprocess the bathymatry since it is simply a two-dimensional field and is easier to deal with. Afterwards the bathymetry is stored at `expt.bathymetry`.

After running this cell, your input directory will contain other bathymetry-related things like the ocean mosaic and mask table too. This defaults to a 10x10 layout which can be updated later.

In [ ]:
expt.setup_bathymetry(
    bathymetry_path='PATH_TO_GEBCO_FILE/GEBCO_2022.nc', 
    longitude_coordinate_name='lon',
    latitude_coordinate_name='lat',
    vertical_coordinate_name='elevation',
    minimum_layers=1
    )

### Check out your domain:

In [ ]:
expt.bathymetry.depth.plot()

##  Step 5: Handle the ocean forcing - where the magic happens

This cuts out and interpolates the initial condition as well as all boundaries (unless you don't pass it boundaries).

The dictionary maps the MOM6 variable names to what they're called in your ocean input file. Notice how for GLORYs, the horizontal dimensions are `x` and `y`, vs `xh`, `yh`, `xq`, `yq` for ACCESS OM2-01. This is because for an 'A' grid type tracers share the grid with velocities so there's no difference.

If one of your segments is land, you can delete its string from the 'boundaries' list. You'll need to update MOM_input to reflect this though so it knows how many segments to look for, and their orientations.

In [ ]:
# Define a mapping from the GLORYS variables and dimensions to the MOM6 ones
ocean_varnames = {"time": "time",
                  "y": "latitude",
                  "x": "longitude",
                  "zl": "depth",
                  "eta": "zos",
                  "u": "uo",
                  "v": "vo",
                  "tracers": {"salt": "so", "temp": "thetao"}
                  }

# Set up the initial condition
expt.initial_condition(
    glorys_path / "ic_unprocessed.nc", # directory where the unprocessed initial condition is stored, as defined earlier
    ocean_varnames,
    gridtype="A"
    )

# Now iterate through our four boundaries 
for i, orientation in enumerate(["south", "north", "west", "east"]):
    expt.rectangular_boundary(
        glorys_path / (orientation + "_unprocessed.nc"),
        ocean_varnames,
        orientation,    # Needs to know the cardinal direction of the boundary
        i + 1,          # Just a number to identify the boundary. Indexes from 1 
        arakawa_grid="A"
        )

## Step 6: Run the FRE tools

This is just a wrapper for the FRE tools needed to make the mosaics and masks for the experiment. The only thing you need to tell it is the processor layout. In this case we're saying that we want a 10 by 10 grid of 100 processors. 

In [ ]:
expt.FRE_tools(layout=(10, 10)) ## Here the tuple defines the processor layout

## Step 7: Set up ERA5 forcing:

Here we assume the ERA5 dataset is stored somewhere on the system we are working. 

Below is a table showing ERA5 characteristics and what needs to be done to sort it out

**Required ERA data**:

Name | ERA filename | era variable name | Units
---|---|---|---
Surface Pressure | sp | sp | Pa 
Surface Temperature | 2t | t2m | K 
Meridional Wind | 10v | v10 | m/s 
Zonal Wind | 10u | u10 | m/s 
Specific Humidity | na | na | kg/kg, calculated from dewpoint temperature
Dewpoint Temperature | 2d | d2m | K


We calculate specific humidity $q$ from dewpoint temperature $T_d$ and surface pressure $P$ via saturation vapour pressure $P_v$.

$$P_v = 10^{8.07131 - \frac{1730.63}{233.426 + T}} \frac{101325}{760} \; \textrm{[Pascal]} $$

$$q = 0.001 \times 0.622  \frac{P_v}{P}$$

In [ ]:
expt.setup_era5("PATH_TO_ERA5_DATA/era5/single-levels/reanalysis")

## Step 8: Modify the default input directory to make a (hopefully) runnable configuration out of the box

This step copies the default directory, and modifies the `MOM_layout` files to match your experiment by inserting the right number of x,y points and cpu layout. If you use Payu to run mom6, set the `using_payu` flag to `True` and an example `config.yaml` file will be copied to your run directory. This still needs to be modified manually to work with your projects, executable etc.

You also need to pass the path to where you git cloned the mom6-regional code. This just ensures that the funciton can find the premade run directories.

In [ ]:
expt.setup_run_directory(surface_forcing = "era5", using_payu = False)

## Step 9: Run and Troubleshoot!

To do this, navigate to your run directory in the terminal, and use your favourite tool to run the experiment on your system. 

Hopefully your model is running. If not, the first thing you should do is reduce the timestep. You can do this by adding `#override DT=XXXX` to your `MOM_override` file. 

If there's strange behaviour on your boundaries, you could play around with the `nudging timescale` (an example is already included in the `MOM_override` file). Sometimes, if your boundary has a lot going on (like all of the eddies spinning off the ACC), it can be hard to avoid these edge effects. This is because the chaotic, submesoscale structures developed within the regional domain won't match those at the boundary. 

Another thing that can go wrong is little bays creating non-advective cells at your boundaries. Keep an eye out for tiny bays where one side is taken up by a boundary segment. You can either fill them in manually, or move your boundary slightly to avoid them